In [1]:
import torch, torchvision
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import os
#import oil.augLayers as augLayers
from oil.model_trainers.classifier import Classifier
from oil.datasetup.datasets import CIFAR10, C10augLayers
from oil.datasetup.dataloaders import getLabLoader
from oil.architectures.smallconv import smallCNN
from oil.utils.utils import cosLr, loader_to
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from oil.lazy.lazy_matrix import LazyMatrix, Lazy
from oil.lazy.lazy_types import LazyAvg
from oil.utils.utils import reusable
from oil.lazy.linalg.VRmethods import GradLoader, oja_grad2,SGHA_grad2,SGD,SVRG, SGHA_grad,oja_subspace_grad,SGHA_subspace_grad2,SGHA_subspace_grad
from oil.logging.lazyLogger import LazyLogger
from oil.lazy.linalg.lanczos import power_method
from oil.lazy.hessian import Hessian, Fisher, autoHvpBatch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import time
import pandas as pd
import os

In [3]:
train_epochs = 100
net_config =        {'numClasses':10,'k':16}
loader_config =     {'amnt_dev':0,'lab_BS':50,'dataseed':0,'num_workers':1}
opt_config =        {'lr':.1, 'momentum':.9, 'weight_decay':1e-4, 'nesterov':True}
sched_config =      {'cycle_length':train_epochs,'cycle_mult':1}
trainer_config =    {'log_args':{'minPeriod':0.1}}
all_hypers = {**net_config,**loader_config,**opt_config,**sched_config,**trainer_config}
trainer_config['log_dir'] = os.path.expanduser('~/tb-experiments/smallcnn/')

def makeTrainer():
    device = torch.device('cuda')
    CNN = smallCNN(**net_config).to(device)
    fullCNN = nn.Sequential(C10augLayers(),CNN)
    trainset, testset = CIFAR10(False, '~/datasets/cifar10/')

    dataloaders = {}
    dataloaders['test'] = DataLoader(testset, batch_size=50,shuffle=False, num_workers=1)
    dataloaders['train'], dataloaders['dev'] = getLabLoader(trainset,**loader_config)
    dataloaders = {k: loader_to(device)(v) for k,v in dataloaders.items()}

    opt_constr = lambda params: optim.SGD(params, **opt_config)
    lr_sched = cosLr(**sched_config)
    return Classifier(fullCNN,dataloaders,opt_constr,lr_sched,**trainer_config,tracked_hypers=all_hypers)

trainer = makeTrainer()
#trainer.train(train_epochs)

Files already downloaded and verified
Files already downloaded and verified
Creating Train, Dev split         with 50000 Train and 0 Dev


In [4]:
#trainer.save_checkpoint()
trainer.load_checkpoint()
print(trainer.getAccuracy(trainer.dataloaders['test']))
print(sum(p.numel() for p in trainer.model.parameters()))

=> loading checkpoint '/home/marc/tb-experiments/smallcnn/checkpoints/c.100.ckpt'
0.8731
45114


In [5]:
'file_writer''all_writers'

'file_writerall_writers'

In [6]:
trainer.logger.__dict__['all_writers']

{'/home/marc/tb-experiments/smallcnn/': <tensorboardX.writer.FileWriter at 0x7f021b970c50>}

In [7]:
import dill

In [8]:
#dill.detect.badtypes(trainer,depth=1)

In [9]:
torch.save(trainer,"__ttt.t",pickle_module=dill)

In [10]:
trainer = torch.load("__ttt.t",pickle_module=dill)

In [11]:
trainer.logger.add_scalars('a',{'b':torch.zeros(10)})

In [ ]:
A = tensorboardX.SummaryWriter()

In [ ]:
torch.save(A,"___ttt.t",pickle_module=dill)

In [ ]:
%debug